In [2]:
from MKLpy.preprocessing import normalization, rescale_01
from sklearn.metrics.pairwise import rbf_kernel as RBF
import os
import pandas as pd
import numpy as np
import pickle as pkl
from MKLpy.metrics.pairwise.misc import homogeneous_polynomial_kernel as HPK_kernel
np.seterr(divide='ignore', invalid='ignore')
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.multiclass import OneVsRestClassifier  # support from multiclass

import time
from sklearn.metrics import accuracy_score, roc_auc_score
from collections import defaultdict
import matplotlib.pyplot as plt
from MKLpy.preprocessing import normalization, rescale_01
from MKLpy.model_selection import cross_val_score, cross_val_predict
import pickle as pkl
###


from MKLpy.metrics import pairwise
from MKLpy.algorithms import AverageMKL, EasyMKL, KOMD#KOMD is not a MKL algorithm but a simple kernel machine like the SVM
#evaluate the solution
from sklearn.metrics import accuracy_score, roc_auc_score 
from sklearn.metrics import roc_curve, auc

In [3]:
c


In [15]:
extPath= '/media/ak/My Passport/Experiment Data'
featuresPath ="/".join((extPath,'features'))
labelsPath ="/".join((extPath,'labels'))
MKLPath = "/".join((extPath,'MKLExpPath'))
symbols = list(set(os.listdir(featuresPath))& set(os.listdir(labelsPath)))
## symbol specific 
symbolIdx =0
symbol =symbols[symbolIdx]
MKLExpPath = "/".join((MKLPath,symbol,'Experiments' )) #symbol is in symbols
MKLSymbolPath = "/".join((MKLPath,symbol)) #symbol is in symbols
MKLKernPath = "/".join((MKLPath,symbol,'Kernels' )) #symbol is in symbols
print(symbol)
os.listdir(MKLSymbolPath)

AAL.L


['CommonLocationsDicts.pkl',
 'Experiments',
 'Kernels',
 'LocDictsListCorrect.pkl']

In [16]:
def forwardDates(idxKey, commonLocs):
    """ return all the forward looking dates for each idxKey we use for training"""
    keys = list(commonLocs.keys())  # this produces a list with 2 keys -first HMM Symbol Date and common Date
    lookAheadKeys = sorted(i for i in keys if i > keys[idxKey])
    return dict((k, commonLocs[k]) for k in lookAheadKeys)


In [17]:
def checkDir(dirLoc):
    print
    if os.path.exists(dirLoc):  # Checks if the dir exists
        print("The directory exists")
    else:
        print("No directory found for " + dirLoc)  # Output if no directory

In [18]:


path =MKLPath
try:
    os.mkdir(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s " % path)

Creation of the directory /media/ak/My Passport/Experiment Data/MKLExpPath failed


In [19]:

try:
    os.mkdir(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s " % path)

Creation of the directory /media/ak/My Passport/Experiment Data/MKLExpPath failed


In [20]:

symbolFeaturesLocation = "/".join((featuresPath,symbol,'MODEL_BASED'))
symbolLabelsLocation = "/".join((labelsPath,symbol,'NON_DIRECTIONAL')) #location of labels for each symbol
symbolHMMDatesList = os.listdir(symbolFeaturesLocation) 
# list of all the MODEL dates we have generated features files for. each #
# each of these dates in symbolFeaturesDates corresponds to a list of dates
# (symbolHMMDatesList = '20170829', '20170710', '20170801', ... ]
symbolLabelsDates = [dateFile.split(".")[0] for dateFile in os.listdir(symbolLabelsLocation)]

In [10]:
# now lets go down into each HMM-model date, and pick all the forward futures (out of sample)
hmmFeatureLocations = {}  # symbol-hmm-model-date index --> this is the indexation in symbolFeaturesDatesList
commonDatesDict = {}  # this is a struct that will contain for each HMM date, the common labels/features- this should
# be used for training and testing
createDate = []  # place holder for the hash key of when the features got created
symbolEachModelFeaturesDates = {}
HMMModelFeaturesLabelsCommon = {}  # location dictionary with 2 keys: HMM Date and Common Date
commonDates = []

LocDictsList = []

In [29]:
symbolHMMDatesList[1]
os.listdir(symbolModelFeaturesDate)[0].split("_")[-2]

'20181227'

In [12]:
hmmDateIdx=1
symbolModelFeaturesDate = os.path.join(symbolFeaturesLocation, symbolHMMDatesList[hmmDateIdx])


In [34]:
for hmmDateIdx, hmmDate in enumerate(sorted(symbolHMMDatesList)):
    symbolModelFeaturesDate = os.path.join(symbolFeaturesLocation, symbolHMMDatesList[hmmDateIdx])
    if len(os.listdir(symbolModelFeaturesDate))!=0:            
        print(symbolModelFeaturesDate)
        create_date = os.listdir(symbolModelFeaturesDate)[0].split("_")[-2]
        # output looks like this: /media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170710
        symbolEachModelFeaturesDates[symbolHMMDatesList[hmmDateIdx]] = [file.split("_")[5] for file in
                                                                        os.listdir(symbolModelFeaturesDate)]
        # output is a dictionary where the keys are the HMM models dates and the values a list of dates - for each HMM
        # date we have a list of features
        for keyHMMDate in sorted(list(symbolEachModelFeaturesDates.keys())):  # for each of the HMM model dates
            print(keyHMMDate)
            common_dates = list(set(symbolEachModelFeaturesDates[keyHMMDate]) & set(symbolLabelsDates))
            print(common_dates)
  
            take the list of feature dates (conditional on HMM model date) + the list of labels -intersection!

        '''we now produce a dict for each HMM model, where each value is a list of common dates and we are key-ed by 
        the HMM Date '''
        commonDatesDict[keyHMMDate] = common_dates
        for commonDate in common_dates:
            '''iterate through all the common dates and figure out the location of each file for labels and 
            features '''
            labelsCommonFileLoc = "/".join((symbolLabelsLocation, ".".join((commonDate, 'csv'))))
            #             comnDateFeatureLocMaster = os.path.join((symbolModelFeaturesDate, commonDate))
            commonDatesFeatureFile = "".join(
                (symbols[symbolIdx], '_3_states_features_date:_', commonDate, "_now:_", create_date, "_.pickle"))
            FeatureFileLoc = os.path.join(symbolModelFeaturesDate, commonDatesFeatureFile)
            checkDir(FeatureFileLoc)
            checkDir(labelsCommonFileLoc)
            conditions = [os.path.exists(FeatureFileLoc), os.path.exists(labelsCommonFileLoc)]
            print(conditions)
            if all(conditions):
                print('all good on Date:', commonDate)
                HMMModelFeaturesLabelsCommon[keyHMMDate, commonDate] = [FeatureFileLoc, labelsCommonFileLoc]
                pkl.dump(HMMModelFeaturesLabelsCommon,
                         open("/".join((MKLSymbolPath, "LocDictsListCorrect.pkl")), "wb"))

            else:
                print('problem on date: ', commonDate)
                continue
            pkl.dump(commonDatesDict,
                     open("/".join((MKLSymbolPath, "CommonLocationsDicts.pkl")), "wb"))
        else:
            continue

    # LocDictsList.extend(commonDatesDict)
    # LocDictsList.extend(HMMModelFeaturesLabelsCommon)

# return LocDictsList


/media/ak/My Passport/Experiment Data/features/AAL.L/MODEL_BASED/20170711
20170116
['20170707', '20170119', '20170717', '20170706', '20170710', '20170822', '20170804', '20170130', '20170724', '20170126', '20170301', '20170802', '20170801', '20170823', '20170705', '20170718', '20170726', '20170123', '20170817', '20170825', '20170816', '20170120', '20170824', '20170831', '20170807', '20170808', '20170127', '20170131', '20170727', '20170703', '20170821', '20170731', '20170124', '20170711', '20170720', '20170117', '20170713', '20170721', '20170818', '20170829', '20170712', '20170815', '20170728', '20170118', '20170811', '20170719', '20170125', '20170725', '20170803', '20170810', '20170830', '20170809', '20170814', '20170704', '20170714']
20170117
['20170707', '20170119', '20170717', '20170706', '20170710', '20170822', '20170804', '20170130', '20170724', '20170126', '20170301', '20170802', '20170801', '20170823', '20170705', '20170718', '20170726', '20170123', '20170817', '20170825', '20170

['20170822', '20170804', '20170724', '20170802', '20170801', '20170823', '20170726', '20170817', '20170825', '20170816', '20170808', '20170824', '20170807', '20170831', '20170727', '20170821', '20170731', '20170721', '20170829', '20170818', '20170815', '20170728', '20170811', '20170725', '20170803', '20170810', '20170830', '20170809', '20170814']
20170721
['20170822', '20170804', '20170724', '20170802', '20170801', '20170823', '20170726', '20170817', '20170825', '20170816', '20170808', '20170824', '20170807', '20170831', '20170727', '20170821', '20170731', '20170829', '20170818', '20170815', '20170728', '20170811', '20170725', '20170803', '20170810', '20170830', '20170809', '20170814']
20170724
['20170822', '20170804', '20170802', '20170801', '20170823', '20170726', '20170817', '20170825', '20170816', '20170808', '20170824', '20170807', '20170831', '20170727', '20170821', '20170731', '20170829', '20170818', '20170815', '20170728', '20170811', '20170725', '20170803', '20170810', '201708

/media/ak/My Passport/Experiment Data/features/AAL.L/MODEL_BASED/20170801
20170116
['20170707', '20170119', '20170717', '20170706', '20170710', '20170822', '20170804', '20170130', '20170724', '20170126', '20170301', '20170802', '20170801', '20170823', '20170705', '20170718', '20170726', '20170123', '20170817', '20170825', '20170816', '20170120', '20170824', '20170831', '20170807', '20170808', '20170127', '20170131', '20170727', '20170703', '20170821', '20170731', '20170124', '20170711', '20170720', '20170117', '20170713', '20170721', '20170818', '20170829', '20170712', '20170815', '20170728', '20170118', '20170811', '20170719', '20170125', '20170725', '20170803', '20170810', '20170830', '20170809', '20170814', '20170704', '20170714']
20170117
['20170707', '20170119', '20170717', '20170706', '20170710', '20170822', '20170804', '20170130', '20170724', '20170126', '20170301', '20170802', '20170801', '20170823', '20170705', '20170718', '20170726', '20170123', '20170817', '20170825', '20170

['20170717', '20170822', '20170804', '20170802', '20170724', '20170801', '20170823', '20170718', '20170726', '20170817', '20170825', '20170816', '20170824', '20170831', '20170807', '20170808', '20170727', '20170821', '20170731', '20170711', '20170720', '20170713', '20170721', '20170712', '20170818', '20170829', '20170815', '20170728', '20170811', '20170719', '20170725', '20170803', '20170810', '20170830', '20170809', '20170814', '20170714']
20170711
['20170717', '20170822', '20170804', '20170724', '20170802', '20170801', '20170823', '20170718', '20170726', '20170817', '20170825', '20170816', '20170824', '20170831', '20170807', '20170808', '20170727', '20170821', '20170731', '20170720', '20170713', '20170721', '20170712', '20170818', '20170829', '20170815', '20170728', '20170811', '20170719', '20170725', '20170803', '20170810', '20170830', '20170809', '20170814', '20170714']
20170712
['20170717', '20170822', '20170804', '20170724', '20170802', '20170801', '20170823', '20170718', '201707

In [33]:
os.listdir('/media/ak/My Passport/Experiment Data/features/AAL.L/MODEL_BASED/20170831')

[]

In [35]:
hmm_features ="/".join((MKLSymbolPath, "LocDictsListCorrect.pkl"))

In [36]:
HMMModelFeaturesLabelsCommon=pkl.load(open(hmm_features, "rb"), encoding='latin1')

In [38]:
count_i=0
for i in range(1,19):

    Xtr,ytr =featureCreation(i, HMMModelFeaturesLabelsCommon)
    if (Xtr.shape[0]==ytr.shape[0]):
        print('Shapes Match- starting training ')
        ##polynomial Kernels ##
        try:
            KLtr = [pairwise.homogeneous_polynomial_kernel(Xtr, degree=d) for d in range(4)]
            ''' Compute RBF Kernels'''
            gamma_range = np.logspace(-9, 3, 13)
            ker_list = [rbf_kernel(Xtr, gamma=g) for g in gamma_range]

        ### and train 3 classifiers ###
            clf = AverageMKL().fit(KLtr,ytr) #a wrapper for averaging kernels
            print(clf.weights) #print the weights of the combination of base kernels
            print ('training EasyMKL...for polynomials and RBF', end='')
            clfEasy = EasyMKL(lam=0.1).fit(KLtr,ytr)#combining kernels with the EasyMKL algorithm
            clfRBF = EasyMKL(lam=0.1).fit(ker_list,ytr)  
            print('------')
            print('finished training')
        except:
            count_i += 1
            print(count_i)
            print(i, "hin failed here!")
            
            continue
            
    else:
        print('Shapes dont match.')
        pass
    print('Average Kernel Testing')
    y_pred = clf.predict(KLtr)                 #predictions
    y_score = clf.decision_function(KLtr)      #rank
    accuracy = accuracy_score(ytr, y_pred)
    fprAverage, tprAverage, thresholdsAverage = roc_curve(ytr.ravel(), y_score.ravel())
    roc_auc = auc(fprAverage, tprAverage)

    print ('Accuracy score: %.3f, roc AUC score: %.3f' % (accuracy, roc_auc))


    ''' Test Linear'''
    print('MKL-Linear Testing')
    y_predMKLLinear = clfEasy.predict(KLtr)                 #predictions
    y_scoreMKLLinear = clfEasy.decision_function(KLtr)  #rank

    accuracy_MKLLinear = accuracy_score(ytr, y_predMKLLinear)
    roc_auc_MKLLinear = roc_auc_score(ytr, y_scoreMKLLinear)
    print ('Accuracy score: %.3f, roc AUC score: %.3f' % (accuracy_MKLLinear, roc_auc_MKLLinear))

    fprLinear, tprLinear, thresholds = roc_curve(ytr.ravel(), y_scoreMKLLinear.ravel())

    roc_auc_Linear = auc(fprLinear, tprLinear)

    # fprMKLLinear, fprMKLLinear, thresholdsRBF =roc_curve(Yte, y_scoreMKLLinear)

    print(roc_auc_Linear)
   

Shapes Match- starting training 
[0.25 0.25 0.25 0.25]
training EasyMKL...for polynomials and RBF------
finished training
Average Kernel Testing
Accuracy score: 0.964, roc AUC score: 0.780
MKL-Linear Testing
Accuracy score: 0.883, roc AUC score: 0.602
0.6018957618068873
Shapes Match- starting training 
[0.25 0.25 0.25 0.25]
training EasyMKL...for polynomials and RBF1
2 hin failed here!


PermissionError: [Errno 13] Permission denied: '/media/ak/My Passport/Experiment Data/features/AAL.L/MODEL_BASED/20170801/AAL.L_3_states_features_date:_20170802_now:_20181227_.pickle'

In [ ]:

print('Average Kernel Testing')
y_pred = clf.predict(KLtr)                 #predictions
y_score = clf.decision_function(KLtr)      #rank
accuracy = accuracy_score(ytr, y_pred)
fprAverage, tprAverage, thresholdsAverage = roc_curve(ytr.ravel(), y_score.ravel())
roc_auc = auc(fprAverage, tprAverage)

print ('Accuracy score: %.3f, roc AUC score: %.3f' % (accuracy, roc_auc))


''' Test Linear'''
print('MKL-Linear Testing')
y_predMKLLinear = clfEasy.predict(KLtr)                 #predictions
y_scoreMKLLinear = clfEasy.decision_function(KLtr)  #rank

accuracy_MKLLinear = accuracy_score(ytr, y_predMKLLinear)
roc_auc_MKLLinear = roc_auc_score(ytr, y_scoreMKLLinear)
print ('Accuracy score: %.3f, roc AUC score: %.3f' % (accuracy_MKLLinear, roc_auc_MKLLinear))

fprLinear, tprLinear, thresholds = roc_curve(ytr.ravel(), y_scoreMKLLinear.ravel())

roc_auc_Linear = auc(fprLinear, tprLinear)

# fprMKLLinear, fprMKLLinear, thresholdsRBF =roc_curve(Yte, y_scoreMKLLinear)

print(roc_auc_Linear)